In [11]:
%run "../../PyUtils/common.ipynb"
import math;
import requests as rq;
from datetime import datetime;

patient_data = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20patient_jlee", checkForDateTime=False);
patient_data[:]

<IPython.core.display.Javascript object>

,pname,gender,age,height,weight,pef_normal,pef_gy_bound,pef_yr_bound,climate_loc,pollution_loc,pefuse_count,pefuse_start,pefuse_end
0,hand2,f,63,154,71,370,296,185,119,1,700,2015-01-01,2016-10-19
1,kimj,f,66,150,57,355,284,176,119,2,365,2015-02-05,2016-02-04
2,kuky,m,48,165,65,590,472,295,119,3,439,2014-11-21,2016-02-02
3,leeh2,f,61,159,63,385,308,192,112,4,592,2015-03-10,2016-10-21
4,kwonc,m,45,172,95,640,512,320,112,4,0,null,null
5,kimk,f,37,160,55,400,320,200,112,4,0,null,null
6,uhy,f,35,149,58,375,300,188,112,5,690,2014-01-01,2015-11-21


In [12]:
url = "http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=";
print(sum(patient_data.pefuse_count[:])*2-690)
pef_data = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20pef_jlee%20WHERE%20pname='" + str(patient_data.pname[0]) + "'%20ORDER%20BY%20ampm,%20date,%20time", checkForDateTime=False);
print(len(pef_data))
pef_data[709:]

4882
1416


,pname,date,ampm,time,pef
709,hand2,2014-12-25,pm,18:31:00,319
710,hand2,2014-12-26,pm,18:50:00,331
711,hand2,2014-12-27,pm,18:58:00,321
712,hand2,2014-12-28,pm,19:03:00,341
...,...,...,...,...,...
1412,hand2,2016-11-27,pm,18:02:00,296
1413,hand2,2016-11-28,pm,18:43:00,290
1414,hand2,2016-11-29,pm,18:51:00,319
1415,hand2,2016-11-30,pm,18:04:00,255


In [13]:
for p_num in range(0, len(patient_data)) :
    if patient_data.pefuse_count[p_num] != 0 :
        print("-!-!-!-"); print(str(patient_data.pname[p_num]));
        calculated_data = pd.DataFrame();
        pef_data = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20pef_jlee%20WHERE%20pname='" + str(patient_data.pname[p_num]) + "'%20ORDER%20BY%20ampm,%20date,%20time", checkForDateTime=False);
        if str(patient_data.pname[p_num]) == 'uhy' :
            pef_data = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20pef_jlee%20WHERE%20pname='" + str(patient_data.pname[p_num]) + "'%20and%20ampm='am'%20ORDER%20BY%20ampm,%20date,%20time", checkForDateTime=False);

        for pef_num in range(0, len(pef_data)) :
            if pef_data.date[pef_num] > patient_data.pefuse_start[p_num] \
            and pef_data.date[pef_num] <= patient_data.pefuse_end[p_num] :
                query1 = "INSERT%20INTO%20calculated_jlee%20("
                query2 = "VALUES%20("

                pollution_data = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q="
                                             "SELECT%20*%20"
                                             "FROM%20pollution_jlee%20"
                                             "WHERE%20location=" + str(patient_data.pollution_loc[p_num]) + "%20"
                                                 "AND%20((date='" + str(pef_data.date[pef_num-1]) + "'%20"
                                                 "AND%20time>'" + str(datetime.strptime(str(pef_data.time[pef_num-1]), '%H:%M:%S') - datetime.strptime("01:00:00", '%H:%M:%S')) + "')%20"
                                                 "OR%20(date='" + str(pef_data.date[pef_num]) + "'%20"
                                                 "AND%20time<'" + str(pef_data.time[pef_num]) + "'))%20"
                                             "ORDER%20BY%20date,%20time", checkForDateTime=False);

                climate_data = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q="
                                             "SELECT%20*%20"
                                             "FROM%20climate_jlee%20"
                                             "WHERE%20location=" + str(patient_data.climate_loc[p_num]) + "%20"
                                                 "AND%20((date='" + str(pef_data.date[pef_num-1]) + "'%20"
                                                 "AND%20time>'" + str(datetime.strptime(str(pef_data.time[pef_num-1]), '%H:%M:%S') - datetime.strptime("01:00:00", '%H:%M:%S')) + "')%20"
                                                 "OR%20(date='" + str(pef_data.date[pef_num]) + "'%20"
                                                 "AND%20time<'" + str(pef_data.time[pef_num]) + "'))%20"
                                             "ORDER%20BY%20date,%20time", checkForDateTime=False);
                
                #print(pollution_data.to_string())
                print(pef_num, end=" ");

                pollution_data_len = len(pollution_data);
                climate_data_len = len(climate_data);

                pef_data_time = datetime.strptime(str(pef_data.time[pef_num]), '%H:%M:%S');
                pollution_data_time = datetime.strptime(str(pollution_data.time[pollution_data_len-1]), '%H:%M:%S');
                climate_data_time = datetime.strptime(str(climate_data.time[climate_data_len-1]), '%H:%M:%S');

                if pef_data.date[pef_num] != pollution_data.date[pollution_data_len-1] \
                or pef_data.date[pef_num] != climate_data.date[climate_data_len-1] \
                or pef_data_time.hour != pollution_data_time.hour \
                or pef_data_time.hour != climate_data_time.hour :
                    if pef_data_time.minute == 0 :
                        if pef_data_time.hour - 1 != climate_data_time.hour \
                        or pef_data_time.hour - 1 != pollution_data_time.hour :
                            continue;
                    else :
                        continue;

                pollution_data = pollution_data[['so2','co','o3','no2','pm10']];
                climate_data = climate_data[['temperaturec','windspeedms','winddirection','precipitationpercent','vaporpressurehpa','dewpointtemperaturec','airpressurehpa','sealevelpressurehpa','groundtemperaturec']];

                start_minute = 1 - ((datetime.strptime(str(pef_data.time[pef_num-1]), '%H:%M:%S')).minute)/60;
                end_minute = (pef_data_time.minute)/60;
                time_window = pollution_data_len - 2;
                time_window = time_window + start_minute;
                time_window = time_window + end_minute;

                calculated_temp = pef_data[pef_num:pef_num+1];

                if pef_data.pef[pef_num] >= patient_data.pef_gy_bound[p_num] :
                    calculated_temp['pef_zone'] = 'G';
                elif pef_data.pef[pef_num] >= patient_data.pef_yr_bound[p_num] :
                    calculated_temp['pef_zone'] = 'Y';
                else :
                    calculated_temp['pef_zone'] = 'R';

                calculated_temp['tm_window'] = time_window;

                for col in pollution_data.columns :
                    calculated_temp[col+'_max'] = max(pollution_data[col]);

                pollution_data.loc[0] = pollution_data.loc[0]*start_minute;
                pollution_data.loc[pollution_data_len-1] = pollution_data.loc[pollution_data_len-1]*end_minute;

                for col in pollution_data.columns :
                    calculated_temp[col+'_sum'] = sum(pollution_data[col]);

                for col in pollution_data.columns :
                    calculated_temp[col+'_avg'] = calculated_temp[col+'_sum']/time_window;

                for col in climate_data.columns :
                    calculated_temp[col] = climate_data.loc[climate_data_len-1][col];

                climate_data = climate_data[['temperaturec', 'airpressurehpa', 'precipitationpercent']];
                climate_data.columns = ['temperature', 'airpressure', 'precipitation'];

                for col in climate_data.columns :
                    calculated_temp[col+'_max'] = max(climate_data[col]);

                for col in climate_data.columns :
                    calculated_temp[col+'_min'] = min(climate_data[col]);

                for col in climate_data.columns :
                    calculated_temp[col+'_diff'] = calculated_temp[col+'_max'] - calculated_temp[col+'_min'];

                #print(calculated_temp);
                for col in range(0, len(calculated_temp.columns)) :
                    query1 = query1 + str(calculated_temp.columns[col]);
                    if calculated_temp.columns[col] == 'pname' or calculated_temp.columns[col] == 'date' \
                    or calculated_temp.columns[col] == 'ampm' or calculated_temp.columns[col] == 'time' \
                    or calculated_temp.columns[col] == 'pef_zone' :
                        query2 = query2 + "'" + str(calculated_temp.loc[pef_num][col]) + "'";
                    else : 
                        query2 = query2 + str(calculated_temp.loc[pef_num][col]);

                    if col == len(calculated_temp.columns) - 1 :
                        query1 = query1 + ")%20";
                        query2 = query2 + ");";
                    else :
                        query1 = query1 + ", ";
                        query2 = query2 + ", ";
                query = query1 + "%20" + query2;
                #print(query);
                response = rq.get(url+query);
                #print(response.text);
                calculated_data = calculated_data.append(calculated_temp, ignore_index = True);

        ###calculated_data.to_csv("./calculated/"+str(patient_data.pname[p_num])+"_calculated.csv", sep = ',', index = False);
        #print(start_minute, end_minute); 
        print(pollution_data)
        print(climate_data)
        print(calculated_data);

-!-!-!-
hand2
9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 27

In [15]:
calculation_temp.loc[9][4]

256

In [4]:
pef_data[218:]

,pname,date,ampm,time,pef
218,uhy,2014-08-07,am,10:43:00,265
219,uhy,2014-08-08,am,09:48:00,296
220,uhy,2014-08-09,am,09:52:00,295
221,uhy,2014-08-10,am,09:37:00,314
...,...,...,...,...,...
686,uhy,2015-11-18,am,09:22:00,307
687,uhy,2015-11-19,am,10:33:00,295
688,uhy,2015-11-20,am,10:32:00,251
689,uhy,2015-11-21,am,11:04:00,234


In [5]:
str(datetime.strptime(str(pef_data.time[218]), '%H:%M:%S') - datetime.strptime("01:00:00", '%H:%M:%S'))

'9:43:00'

In [39]:
patient_data = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20patient_jlee", checkForDateTime=False);
for i in range(0, len(patient_data)) :
    url = "http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20calculated_jlee%20WHERE%20pname='"+str(patient_data.pname[i])+"'%20ORDER%20BY%20date,%20time";
    dfOriginal = LoadDataSet(url, checkForDateTime=False);
    display(dfOriginal);

,pname,date,ampm,time,pef,pef_zone,tm_window,so2_max,co_max,o3_max,no2_max,pm10_max,so2_sum,co_sum,o3_sum,no2_sum,pm10_sum,so2_avg,co_avg,o3_avg,no2_avg,pm10_avg,temperaturec,windspeedms,winddirection,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperature_max,airpressure_max,precipitation_max,temperature_min,airpressure_min,precipitation_min,temperature_diff,airpressure_diff,precipitation_diff
0,hand2,2015-01-02,am,06:24:00,279,Y,22.700,0.004,0.700,0.025,0.024,145.000,0.078,12.790,0.428,0.300,990.300,0.003,0.563,0.019,0.013,43.626,-6.800,1.200,340.000,58.000,2.100,-13.600,1020.500,1025.000,-4.800,-3.900,1022.200,63.000,-8.500,1019.600,35.000,4.600,2.600,28.000
1,hand2,2015-01-03,am,07:19:00,251,Y,24.917,0.009,0.900,0.026,0.035,54.000,0.125,17.045,0.299,0.584,886.283,0.005,0.684,0.012,0.023,35.570,-9.600,0.600,180.000,71.000,2.100,-13.800,1022.700,1027.200,-6.900,-0.800,1023.400,71.000,-9.600,1019.900,31.000,8.800,3.500,40.000
2,hand2,2015-01-04,am,07:46:00,260,Y,24.450,0.013,1.000,0.015,0.039,52.000,0.174,16.975,0.180,0.797,857.550,0.007,0.694,0.007,0.033,35.074,0.700,1.600,70.000,67.000,4.300,-4.700,1013.600,1017.900,-0.700,1.900,1023.400,71.000,-9.600,1013.200,38.000,11.500,10.200,33.000
3,hand2,2015-01-05,am,06:08:00,273,Y,22.367,0.009,1.300,0.021,0.065,75.000,0.137,22.300,0.106,1.078,1248.967,0.006,0.997,0.005,0.048,55.841,-2.800,0.000,0.000,100.000,5.000,-2.800,1016.200,1020.600,-2.800,7.600,1017.500,100.000,-2.800,1012.900,59.000,10.400,4.600,41.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,hand2,2016-10-16,am,07:17:00,260,Y,25.283,0.004,1.100,0.045,0.059,105.000,0.086,19.070,0.332,1.090,1863.967,0.003,0.754,0.013,0.043,73.723,16.700,1.000,110.000,77.000,14.600,12.600,1017.500,1021.600,15.200,26.100,1023.500,100.000,11.200,1017.500,35.000,14.900,6.000,65.000
629,hand2,2016-10-17,am,06:11:00,292,Y,22.900,0.004,0.700,0.028,0.037,74.000,0.073,13.440,0.279,0.642,1261.567,0.003,0.587,0.012,0.028,55.090,15.000,0.800,340.000,97.000,16.500,14.500,1013.400,1017.500,15.300,19.400,1017.600,100.000,15.000,1012.600,60.000,4.400,5.000,40.000
630,hand2,2016-10-18,am,06:00:00,286,Y,22.817,0.004,0.900,0.055,0.039,72.000,0.066,12.690,0.453,0.625,1205.467,0.003,0.556,0.020,0.027,52.833,11.900,0.600,250.000,100.000,13.800,11.800,1016.100,1020.200,12.900,22.400,1016.200,100.000,11.900,1013.000,55.000,10.500,3.200,45.000
631,hand2,2016-10-19,am,06:28:00,311,G,24.467,0.007,1.000,0.032,0.064,104.000,0.118,18.420,0.199,1.045,1752.667,0.005,0.753,0.008,0.043,71.635,11.700,0.000,0.000,100.000,13.700,11.600,1014.900,1019.000,11.900,20.800,1018.100,100.000,11.000,1014.900,61.000,9.800,3.200,39.000


,pname,date,ampm,time,pef,pef_zone,tm_window,so2_max,co_max,o3_max,no2_max,pm10_max,so2_sum,co_sum,o3_sum,no2_sum,pm10_sum,so2_avg,co_avg,o3_avg,no2_avg,pm10_avg,temperaturec,windspeedms,winddirection,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperature_max,airpressure_max,precipitation_max,temperature_min,airpressure_min,precipitation_min,temperature_diff,airpressure_diff,precipitation_diff
0,kimj,2015-02-06,am,06:09:00,85,R,24.367,0.018,1.100,0.041,0.057,125.000,0.237,19.578,0.605,0.920,2115.567,0.010,0.803,0.025,0.038,86.822,-4.700,0.500,110.000,93.000,4.000,-5.600,1021.900,1026.300,-2.800,4.700,1021.900,93.000,-4.700,1018.000,42.000,9.400,3.900,51.000
1,kimj,2015-02-07,am,06:34:00,123,R,24.417,0.012,1.300,0.033,0.069,81.000,0.175,18.618,0.267,1.215,1262.033,0.007,0.763,0.011,0.050,51.687,-1.900,0.800,200.000,91.000,4.900,-3.100,1018.900,1023.300,-1.300,6.100,1023.000,94.000,-4.700,1018.900,44.000,10.800,4.100,50.000
2,kimj,2015-02-08,am,09:26:00,128,R,26.867,0.012,1.300,0.040,0.072,175.000,0.171,13.707,0.712,0.599,2000.300,0.006,0.510,0.026,0.022,74.453,-9.800,4.600,320.000,33.000,1.000,-23.000,1022.700,1027.200,-4.100,5.100,1022.700,91.000,-10.400,1015.500,32.000,15.500,7.200,59.000
3,kimj,2015-02-09,am,06:13:00,97,R,20.783,0.009,0.400,0.033,0.015,166.000,0.091,6.657,0.563,0.144,1573.650,0.004,0.320,0.027,0.007,75.717,-11.300,1.000,290.000,45.000,13.042,-20.900,1026.600,1031.200,-6.100,-5.700,1026.600,46.000,-11.300,1022.300,29.000,5.600,4.300,17.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,kimj,2016-10-03,am,11:53:00,148,R,30.100,0.003,0.600,0.030,0.038,40.000,0.074,12.295,0.531,0.612,586.200,0.002,0.408,0.018,0.020,19.475,23.400,2.000,290.000,85.000,24.400,20.700,1006.000,1009.900,24.500,23.600,1014.000,100.000,16.900,1003.200,83.000,6.700,10.800,17.000
606,kimj,2016-10-04,am,11:25:00,189,Y,23.533,0.003,0.700,0.046,0.045,42.000,0.062,10.643,0.511,0.656,671.467,0.003,0.452,0.022,0.028,28.533,24.200,3.500,90.000,55.000,16.500,14.500,1012.100,1016.000,26.700,27.700,1012.700,100.000,14.200,1005.200,54.000,13.500,7.500,46.000
607,kimj,2016-10-05,am,05:59:00,190,Y,18.567,0.003,0.600,0.051,0.059,41.000,0.038,8.485,0.501,0.517,562.667,0.002,0.457,0.027,0.028,30.305,20.000,3.300,70.000,65.000,15.200,13.200,1006.500,1010.500,18.900,28.200,1012.100,67.000,19.900,1006.500,43.000,8.300,5.600,24.000
608,kimj,2016-10-06,am,05:47:00,135,R,23.800,0.003,0.600,0.050,0.039,50.000,0.061,10.898,0.730,0.492,841.883,0.003,0.458,0.031,0.021,35.373,15.000,2.900,320.000,84.000,14.300,12.300,1015.100,1019.200,15.400,26.500,1015.100,95.000,15.000,1003.900,53.000,11.500,11.200,42.000


,pname,date,ampm,time,pef,pef_zone,tm_window,so2_max,co_max,o3_max,no2_max,pm10_max,so2_sum,co_sum,o3_sum,no2_sum,pm10_sum,so2_avg,co_avg,o3_avg,no2_avg,pm10_avg,temperaturec,windspeedms,winddirection,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperature_max,airpressure_max,precipitation_max,temperature_min,airpressure_min,precipitation_min,temperature_diff,airpressure_diff,precipitation_diff
0,kuky,2014-11-22,am,10:10:00,453,Y,27.000,0.007,0.500,0.037,0.032,137.000,0.118,9.983,0.523,0.619,2135.667,0.004,0.370,0.019,0.023,79.099,9.500,0.800,160.000,99.000,11.700,9.300,1020.900,1025.100,9.000,15.500,1020.900,99.000,2.900,1017.100,46.000,12.600,3.800,53.000
1,kuky,2014-11-23,am,08:42:00,430,Y,22.533,0.003,0.600,0.019,0.032,95.000,0.068,10.283,0.126,0.517,1474.400,0.003,0.456,0.006,0.023,65.432,6.500,1.700,70.000,100.000,9.600,6.400,1023.600,1027.800,7.000,10.700,1023.600,100.000,4.800,1018.900,91.000,5.900,4.700,9.000
2,kuky,2014-11-24,am,09:01:00,464,Y,24.317,0.004,0.700,0.021,0.030,132.000,0.079,13.062,0.180,0.534,2414.300,0.003,0.537,0.007,0.022,99.286,9.400,0.300,0.000,74.000,8.700,4.900,1018.900,1023.100,8.300,11.700,1023.800,100.000,6.500,1018.300,67.000,5.200,5.500,33.000
3,kuky,2014-11-25,am,08:35:00,418,Y,23.567,0.005,0.900,0.006,0.035,155.000,0.089,13.813,0.090,0.634,2408.300,0.004,0.586,0.004,0.027,102.191,4.800,1.000,270.000,96.000,8.200,4.200,1014.400,1018.600,3.400,12.300,1018.900,96.000,4.000,1013.300,42.000,8.300,5.600,54.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,kuky,2016-08-20,am,09:01:00,470,Y,25.750,0.018,0.700,0.128,0.040,98.000,0.205,10.798,1.529,0.592,1827.417,0.008,0.419,0.059,0.023,70.968,29.800,2.600,110.000,55.000,22.900,19.700,1003.700,1007.500,32.200,33.900,1003.800,87.000,27.200,1002.000,55.000,6.700,1.800,32.000
638,kuky,2016-08-21,am,07:10:00,495,G,22.150,0.016,0.700,0.124,0.031,90.000,0.147,9.128,1.563,0.365,1277.450,0.007,0.412,0.071,0.016,57.673,28.000,2.600,110.000,59.000,22.200,19.200,1002.800,1006.600,28.100,35.200,1003.900,77.000,28.000,1000.500,41.000,7.200,3.400,36.000
639,kuky,2016-08-22,am,07:38:00,536,G,24.467,0.038,0.500,0.080,0.037,55.000,0.226,7.483,1.135,0.370,761.833,0.009,0.306,0.046,0.015,31.138,26.300,0.700,180.000,83.000,28.300,23.100,1003.300,1007.200,26.700,35.900,1003.400,90.000,25.300,1000.900,40.000,10.600,2.500,50.000
640,kuky,2016-08-23,am,07:07:00,516,G,23.483,0.008,0.500,0.098,0.027,60.000,0.120,7.007,1.081,0.412,941.600,0.005,0.298,0.046,0.018,40.097,27.100,2.800,110.000,74.000,26.400,22.000,1002.900,1006.700,27.700,34.000,1003.600,91.000,26.300,1000.600,53.000,7.700,3.000,38.000


,pname,date,ampm,time,pef,pef_zone,tm_window,so2_max,co_max,o3_max,no2_max,pm10_max,so2_sum,co_sum,o3_sum,no2_sum,pm10_sum,so2_avg,co_avg,o3_avg,no2_avg,pm10_avg,temperaturec,windspeedms,winddirection,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperature_max,airpressure_max,precipitation_max,temperature_min,airpressure_min,precipitation_min,temperature_diff,airpressure_diff,precipitation_diff
0,leeh2,2015-03-11,am,09:00:00,253,Y,22.900,0.017,0.700,0.034,0.048,54.000,0.133,10.730,0.619,0.450,719.700,0.006,0.469,0.027,0.020,31.428,-1.200,2.600,200.000,63.000,3.500,-7.300,1013.000,1022.100,-1.100,-0.100,1018.500,65.000,-5.000,1013.000,38.000,4.900,5.500,27.000
1,leeh2,2015-03-12,am,09:01:00,256,Y,24.017,0.010,0.900,0.041,0.047,121.000,0.131,11.912,0.718,0.547,1124.750,0.005,0.496,0.030,0.023,46.832,1.200,2.500,270.000,53.000,3.500,-7.300,1012.600,1021.600,0.000,5.500,1013.000,73.000,-1.400,1007.600,50.000,6.900,5.400,23.000
2,leeh2,2015-03-13,am,09:30:00,254,Y,24.483,0.007,0.800,0.040,0.047,90.000,0.131,12.838,0.675,0.653,1066.750,0.005,0.524,0.028,0.027,43.570,3.600,2.100,360.000,90.000,7.100,2.100,1010.600,1019.500,4.600,4.900,1012.600,99.000,1.200,1008.100,41.000,3.700,4.500,58.000
3,leeh2,2015-03-14,am,09:11:00,222,Y,23.683,0.007,0.800,0.044,0.048,60.000,0.128,10.497,0.769,0.596,1027.667,0.005,0.443,0.032,0.025,43.392,2.300,1.900,20.000,51.000,3.700,-6.800,1017.100,1026.100,1.900,8.700,1017.100,90.000,0.100,1010.300,37.000,8.600,6.800,53.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,leeh2,2016-10-18,am,09:12:00,275,Y,23.983,0.006,0.800,0.048,0.060,95.000,0.104,12.830,0.520,0.750,1504.417,0.004,0.535,0.022,0.031,62.728,13.300,1.100,20.000,85.000,12.900,10.800,1013.200,1021.800,14.000,20.700,1013.200,98.000,12.400,1008.800,76.000,8.300,4.400,22.000
588,leeh2,2016-10-19,am,09:31:00,258,Y,24.317,0.010,1.100,0.026,0.076,116.000,0.114,19.005,0.205,1.359,2154.933,0.005,0.782,0.008,0.056,88.620,17.000,0.400,0.000,93.000,17.900,15.800,1011.000,1019.400,17.000,21.200,1013.400,98.000,13.300,1010.200,67.000,7.900,3.200,31.000
589,leeh2,2016-10-20,am,09:02:00,271,Y,23.517,0.011,0.900,0.061,0.075,121.000,0.128,16.858,0.429,1.262,1979.567,0.005,0.717,0.018,0.054,84.177,15.700,2.700,20.000,88.000,15.700,13.700,1012.200,1020.700,16.000,21.600,1012.200,98.000,14.400,1008.200,71.000,7.200,4.000,27.000
590,leeh2,2016-10-21,am,09:03:00,303,Y,24.017,0.006,0.800,0.050,0.075,122.000,0.096,13.497,0.570,0.877,1633.650,0.004,0.562,0.024,0.036,68.022,17.600,5.000,140.000,73.000,14.600,12.600,1012.500,1020.900,16.700,24.200,1012.500,88.000,15.700,1009.000,52.000,8.500,3.500,36.000


,pname,date,ampm,time,pef,pef_zone,tm_window,so2_max,co_max,o3_max,no2_max,pm10_max,so2_sum,co_sum,o3_sum,no2_sum,pm10_sum,so2_avg,co_avg,o3_avg,no2_avg,pm10_avg,temperaturec,windspeedms,winddirection,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperature_max,airpressure_max,precipitation_max,temperature_min,airpressure_min,precipitation_min,temperature_diff,airpressure_diff,precipitation_diff
0,kwonc,2014-01-02,am,06:58:00,486,Y,20.850,0.015,0.700,0.025,0.039,144.000,0.214,9.918,0.378,0.458,2329.300,0.010,0.476,0.018,0.022,111.717,0.800,2.300,320.000,67.000,4.300,-4.600,1012.000,1020.600,-0.700,7.200,1012.000,82.000,0.700,1004.600,52.000,6.500,7.400,30.000
1,kwonc,2014-01-03,am,07:35:00,486,Y,24.617,0.014,1.700,0.018,0.087,93.000,0.224,26.040,0.155,1.416,1623.933,0.009,1.058,0.006,0.058,65.969,0.500,1.200,140.000,60.000,3.800,-6.300,1009.200,1017.800,-0.100,5.700,1015.100,70.000,0.300,1009.200,37.000,5.400,5.900,33.000
2,kwonc,2014-01-04,am,07:35:00,505,Y,24.000,0.012,1.600,0.019,0.095,113.000,0.191,17.217,0.211,0.986,1635.167,0.008,0.717,0.009,0.041,68.132,-2.700,3.500,20.000,62.000,3.100,-8.900,1010.700,1019.400,-1.600,6.600,1010.700,96.000,-2.700,1006.400,56.000,9.300,4.300,40.000
3,kwonc,2014-01-05,am,07:35:00,399,Y,24.000,0.009,0.800,0.021,0.040,62.000,0.125,10.500,0.309,0.604,1155.750,0.005,0.438,0.013,0.025,48.156,-3.500,3.200,50.000,58.000,2.800,-10.500,1014.300,1023.100,-2.000,2.700,1014.300,62.000,-3.500,1009.800,50.000,6.200,4.500,12.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,kwonc,2015-08-30,am,07:02:00,394,Y,24.000,0.014,0.500,0.075,0.032,44.000,0.127,9.397,0.975,0.438,780.067,0.005,0.392,0.041,0.018,32.503,22.100,0.000,0.000,98.000,26.000,21.700,1003.000,1011.200,21.200,28.000,1003.000,98.000,21.600,999.500,67.000,6.400,3.500,31.000
606,kwonc,2015-08-31,am,07:02:00,445,Y,24.000,0.011,0.700,0.074,0.034,51.000,0.134,10.510,0.942,0.345,906.233,0.006,0.438,0.039,0.014,37.760,21.900,1.400,360.000,98.000,25.600,21.500,1004.400,1012.600,22.100,28.300,1004.400,98.000,21.900,1001.300,65.000,6.400,3.100,33.000
607,kwonc,2015-09-01,am,06:35:00,418,Y,23.550,0.005,0.700,0.070,0.057,58.000,0.090,11.610,0.893,0.672,904.967,0.004,0.493,0.038,0.029,38.427,23.100,1.600,20.000,98.000,27.600,22.700,1002.400,1010.600,22.000,28.500,1004.500,98.000,21.900,1001.900,65.000,6.600,2.600,33.000
608,kwonc,2015-09-02,am,06:26:00,422,Y,23.850,0.004,0.500,0.053,0.048,53.000,0.074,9.297,0.723,0.391,474.800,0.003,0.390,0.030,0.016,19.908,23.400,3.500,180.000,98.000,28.100,23.000,1003.500,1011.700,22.400,30.200,1003.500,98.000,22.500,1000.600,54.000,7.700,2.900,44.000


,pname,date,ampm,time,pef,pef_zone,tm_window,so2_max,co_max,o3_max,no2_max,pm10_max,so2_sum,co_sum,o3_sum,no2_sum,pm10_sum,so2_avg,co_avg,o3_avg,no2_avg,pm10_avg,temperaturec,windspeedms,winddirection,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperature_max,airpressure_max,precipitation_max,temperature_min,airpressure_min,precipitation_min,temperature_diff,airpressure_diff,precipitation_diff
0,kimk,2014-01-02,am,11:13:00,134,R,24.000,0.015,0.700,0.025,0.040,144.000,0.244,11.922,0.389,0.582,2473.650,0.010,0.497,0.016,0.024,103.069,1.900,2.000,340.000,43.000,3.000,-9.300,1015.100,1023.700,6.200,7.200,1015.100,82.000,0.300,1004.600,43.000,6.900,10.500,39.000
1,kimk,2014-01-03,am,07:54:00,161,R,20.683,0.012,1.700,0.018,0.087,93.000,0.188,23.952,0.122,1.296,1381.433,0.009,1.158,0.006,0.063,66.790,0.500,1.200,140.000,60.000,3.800,-6.300,1009.200,1017.800,-0.100,5.700,1015.100,70.000,0.300,1009.200,37.000,5.400,5.900,33.000
2,kimk,2014-01-04,am,07:30:00,178,R,23.600,0.012,1.600,0.019,0.095,113.000,0.188,16.740,0.209,0.959,1610.000,0.008,0.709,0.009,0.041,68.220,-2.700,3.500,20.000,62.000,3.100,-8.900,1010.700,1019.400,-1.600,6.600,1010.700,96.000,-2.700,1006.400,56.000,9.300,4.300,40.000
3,kimk,2014-01-05,am,09:26:00,156,R,25.933,0.009,0.800,0.021,0.040,62.000,0.135,11.417,0.328,0.653,1253.100,0.005,0.440,0.013,0.025,48.320,-2.900,2.700,20.000,55.000,2.700,-10.600,1015.200,1024.000,-0.400,2.700,1015.200,62.000,-3.500,1009.800,50.000,6.200,5.400,12.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,kimk,2016-10-18,am,05:42:00,115,R,22.433,0.006,0.800,0.048,0.060,95.000,0.098,12.120,0.516,0.685,1469.833,0.004,0.540,0.023,0.031,65.520,13.000,3.200,20.000,79.000,11.800,9.400,1011.700,1020.300,11.200,20.700,1012.000,98.000,13.000,1008.800,76.000,7.700,3.200,22.000
1021,kimk,2016-10-19,am,07:17:00,118,R,25.583,0.010,1.100,0.026,0.076,116.000,0.120,18.632,0.214,1.363,2129.800,0.005,0.728,0.008,0.053,83.250,15.200,0.400,0.000,97.000,16.700,14.700,1010.400,1018.900,13.500,21.200,1013.400,98.000,12.400,1010.200,67.000,8.800,3.200,31.000
1022,kimk,2016-10-20,am,06:37:00,157,R,23.333,0.011,1.100,0.061,0.075,121.000,0.127,17.282,0.428,1.263,1984.300,0.005,0.741,0.018,0.054,85.041,14.700,2.500,20.000,98.000,16.300,14.300,1011.000,1019.500,12.600,21.600,1011.000,98.000,14.700,1008.200,71.000,6.900,2.800,27.000
1023,kimk,2016-10-21,am,09:12:00,155,R,26.583,0.006,0.800,0.050,0.075,122.000,0.106,15.387,0.582,1.011,1859.800,0.004,0.579,0.022,0.038,69.961,17.600,5.000,140.000,73.000,14.600,12.600,1012.500,1020.900,16.700,24.200,1012.500,98.000,14.400,1009.000,52.000,9.800,3.500,46.000


,pname,date,ampm,time,pef,pef_zone,tm_window,so2_max,co_max,o3_max,no2_max,pm10_max,so2_sum,co_sum,o3_sum,no2_sum,pm10_sum,so2_avg,co_avg,o3_avg,no2_avg,pm10_avg,temperaturec,windspeedms,winddirection,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperature_max,airpressure_max,precipitation_max,temperature_min,airpressure_min,precipitation_min,temperature_diff,airpressure_diff,precipitation_diff
0,uhy,2014-01-02,am,09:58:00,368,G,24.450,0.015,1.300,0.026,0.044,108.000,0.254,16.363,0.419,0.630,1948.783,0.010,0.669,0.017,0.026,79.705,0.300,1.900,20.000,48.000,3.000,-9.400,1014.000,1022.600,-0.100,7.200,1014.000,82.000,0.300,1004.600,48.000,6.900,9.400,34.000
1,uhy,2014-01-03,am,09:30:00,310,G,23.533,0.011,2.200,0.023,0.077,90.000,0.225,23.727,0.192,1.136,1180.400,0.010,1.008,0.008,0.048,50.159,1.200,0.900,160.000,62.000,4.200,-5.200,1009.000,1017.600,0.300,5.700,1015.100,70.000,0.300,1008.900,37.000,5.400,6.200,33.000
2,uhy,2014-01-04,am,09:31:00,326,G,24.017,0.014,2.200,0.015,0.084,117.000,0.215,18.910,0.191,0.952,1306.317,0.009,0.787,0.008,0.040,54.392,-2.700,4.500,20.000,59.000,3.000,-9.600,1011.600,1020.300,-0.800,6.600,1011.600,96.000,-3.000,1006.400,56.000,9.600,5.200,40.000
3,uhy,2014-01-05,am,11:01:00,328,G,25.500,0.007,0.800,0.022,0.042,58.000,0.170,15.402,0.281,0.679,1089.650,0.007,0.604,0.011,0.027,42.731,-0.800,2.600,340.000,53.000,3.000,-9.200,1015.600,1024.300,3.700,2.700,1015.600,62.000,-3.500,1009.800,50.000,6.200,5.800,12.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,uhy,2015-11-18,am,09:22:00,307,G,24.000,0.006,1.100,0.048,0.040,50.000,0.115,13.937,0.567,0.690,568.000,0.005,0.581,0.024,0.029,23.667,13.000,4.100,110.000,80.000,11.900,9.600,1011.300,1019.900,11.200,16.900,1012.200,97.000,11.200,1009.700,66.000,5.700,2.500,31.000
686,uhy,2015-11-19,am,10:33:00,295,Y,25.183,0.005,0.600,0.031,0.033,24.000,0.095,10.100,0.443,0.691,220.567,0.004,0.401,0.018,0.027,8.758,9.400,3.700,140.000,84.000,9.900,6.800,1015.200,1023.900,10.500,13.000,1015.800,98.000,7.500,1010.400,77.000,5.500,5.400,21.000
687,uhy,2015-11-20,am,10:32:00,251,Y,23.983,0.008,1.000,0.018,0.032,47.000,0.102,14.153,0.189,0.664,718.267,0.004,0.590,0.008,0.028,29.949,9.900,1.100,360.000,98.000,11.900,9.500,1015.700,1024.500,11.700,11.300,1015.800,98.000,8.700,1012.400,84.000,2.600,3.400,14.000
688,uhy,2015-11-21,am,11:04:00,234,Y,24.533,0.008,1.300,0.019,0.038,89.000,0.126,20.507,0.166,0.705,1316.000,0.005,0.836,0.007,0.029,53.641,9.000,2.100,340.000,68.000,7.700,3.300,1019.300,1028.100,14.000,13.300,1019.500,98.000,5.800,1014.400,68.000,7.500,5.100,30.000
